<a href="https://colab.research.google.com/github/AI4Bharat/indicTrans/blob/main/indictrans_fairseq_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create a seperate folder to store everything
!mkdir testing
%cd testing

/home/maharaj/repositories/experiments/indicTrans/testing


In [2]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 694 (delta 273), reused 272 (delta 194), pack-reused 297
Receiving objects: 100% (694/694), 2.65 MiB | 4.28 MiB/s, done.
Resolving deltas: 100% (400/400), done.
/home/maharaj/repositories/experiments/indicTrans/testing/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (721/721), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126

In [3]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 34534, done.
remote: Total 34534 (delta 0), reused 0 (delta 0), pack-reused 34534
Receiving objects: 100% (34534/34534), 24.06 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (25109/25109), done.
/home/maharaj/repositories/experiments/indicTrans/testing/fairseq
Processing /home/maharaj/repositories/experiments/indicTrans/testing/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 23.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:03
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
/home/maharaj/repositories/experiments/indicTrans/testing


In [5]:
# add fairseq folder to python path
import os
# os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

/home/maharaj/anaconda3/envs/pyt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maharaj/anaconda3/envs/pyt/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-18 10:59:41 | WARNING | xformers | Triton is not available, some optimizations will not be enabled.
This is just a warning: No module named 'triton'


In [8]:
# download the indictrans model


# downloading the indic-en model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
!unzip indic2en.zip

# downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
!unzip en2indic.zip

# # downloading the indic-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.tar
!tar -xvf m2m.tar

%cd indicTrans

--2023-03-18 11:37:58--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.tar
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 101.53.152.33, 101.53.136.18, 164.52.210.96, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|101.53.152.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5236582400 (4.9G) [application/x-tar]
Saving to: ‘m2m.tar’

m2m.tar               2%[                    ] 124.73M  6.74MB/s    in 3m 23s  

2023-03-18 11:41:22 (629 KB/s) - Connection closed at byte 130792973. Retrying.

--2023-03-18 11:41:23--  (try: 2)  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.tar
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|101.53.152.33|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 5236582400 (4.9G)

In [12]:
# creating a text file and adding en sentences we can use for testing the model
!touch en_sentences.txt
!echo 'This bicycle is too small for you !!' >> en_sentences.txt
!echo "I will directly meet you at the airport." >> en_sentences.txt
!echo 'If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue' >> en_sentences.txt

In [ ]:
# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to tamil
!bash joint_translate.sh en_sentences.txt ta_outputs.txt 'en' 'ta' '../en-indic'

Wed Jun 9 15:18:01 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 71.78it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [11]:
!cat ta_outputs.txt

In [ ]:
# Similarly, we can translate the english sentences to hindi
!bash joint_translate.sh en_sentences.txt hi_outputs.txt 'en' 'hi' '../en-indic'

Wed Jun 9 15:21:31 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 88.59it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
!cat hi_outputs.txt

यह साइकिल तुम्हारे लिए बहुत छोटी है!
मैं आपसे एयरपोर्ट पर ही मिलने वाला हूं।
यदि आपके समुदाय में कोविड-19 फैल रहा है, तो कुछ सरल सावधानियां बरतें, जैसे शारीरिक दूरी बनाए रखना, मास्क पहनना, कमरों को अच्छी तरह से हवादार रखना, भीड़ से बचना, अपने हाथों को साफ करना और कोहनी या ऊतक को मोड़कर खांसते हुए सुरक्षित रहें


In [ ]:
# creating a text file and adding hi sentences we can use for testing the model
!touch hi_sentences.txt
!echo 'तुम आज सुबह यहाँ क्यों आए?' >> hi_sentences.txt
!echo "मेरे परिवार में हर कोई जल्दी उठता है।" >> hi_sentences.txt
!echo ' स्वास्थ्य और परिवार कल्याण मंत्रालय द्वारा प्रदान की गई जानकारी और सलाह को सावधानी व सही तरीके से पालन कर वायरस के स्थानीय प्रसार को रोका जा सकता है।' >> hi_sentences.txt

!touch ta_sentences.txt
!echo 'அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது' >> ta_sentences.txt
!echo "இது எங்கே இருக்கு என்று என்னால் கண்டுபிடிக்க முடியவில்லை." >> ta_sentences.txt
!echo 'உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.' >> ta_sentences.txt

In [ ]:
# here we are translating the english sentences to hindi
!bash joint_translate.sh hi_sentences.txt en_outputs.txt 'hi' 'en' '../indic-en'

Wed Jun 9 15:24:43 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 74.90it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

Why did you come here this morning?
Everyone in my family gets up early.
The local spread of the virus can be curbed by following the information and advice provided by the Ministry of Health and Family Welfare in a careful and correct manner.


In [ ]:
# here we are translating the english sentences to tamil
!bash joint_translate.sh ta_sentences.txt en_outputs.txt 'ta' 'en' '../indic-en'

Wed Jun 9 15:28:05 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 72.92it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

He seems to know us.
I couldnt find it anywhere.
If someone in your neighbourhood develops these symptoms, staying at home can help prevent the spread of the coronavirus infection.


In [ ]:
# we just rename the m2m_joint_vocab file here as joint_translate uses bpe_codes.32k.SRC
mv ../m2m/vocab/bpe_codes.32k.SRC_TGT ../m2m/vocab/bpe_codes.32k.SRC

In [ ]:
# here we are using the indic2indic model for translating the hindi sentences to tamil
!bash joint_translate.sh hi_sentences.txt ta_outputs.txt 'hi' 'ta' '../m2m'

Wed Jun 9 15:39:26 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 63.53it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
 ! cat hi_sentences.txt # the hindi inputs

तुम आज सुबह यहाँ क्यों आए?
मेरे परिवार में हर कोई जल्दी उठता है।
 स्वास्थ्य और परिवार कल्याण मंत्रालय द्वारा प्रदान की गई जानकारी और सलाह को सावधानी व सही तरीके से पालन कर वायरस के स्थानीय प्रसार को रोका जा सकता है।


In [ ]:
! cat ta_outputs.txt # the tamil outputs

ஏன் இன்று காலையில் வந்தீர்கள்?
எனது குடும்பத்தில் உள்ள ஒவ்வொருவரும் விரைவில் எழுவார்கள்.
மத்திய சுகாதாரம் மற்றும் குடும்ப நல அமைச்சகத்தின் அறிவுறுத்தல்கள் மற்றும் தகவல்களைப் பின்பற்றுவதன் மூலம், உள்ளூர் அளவில் வைரஸ் பரவுவதைத் தடுக்க முடியும்.


In [ ]:
# here we are using the indic2indic model for translating the hindi sentences to tamil (same as above with reversing the direction)
!bash joint_translate.sh ta_sentences.txt hi_outputs.txt 'ta' 'hi' '../m2m'

Wed Jun 9 15:45:53 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 82.25it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat ta_sentences.txt # the tamil inputs

அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது
இது எங்கே இருக்கு என்று என்னால் கண்டுபிடிக்க முடியவில்லை.
உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.


In [ ]:
! cat hi_outputs.txt   # the hi outputs

ऐसा लगता है कि वह हमें जानता है।
मुझे पता नहीं था कि यह कहां है।
अगर आपके आस-पास के किसी व्यक्ति में ऐसे लक्षण दिखाई देते हैं, तो घर पर रहने से कोरोना वायरस को फैलने से रोकने में मदद मिलेगी।


In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command

# bash compute_bleu.sh pred_fname ref_fname src_lang tgt_lang
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language